In [1]:
# MTF booking 2022
# 2021 10 22 Остоженка. Делаю отчет по тепл стартующим в 2022 году

# 2021-10-29

# 2021-11-02 переменную со сводной табл прямо в тело письма

In [2]:
#работаю в директории:
import os

os.chdir('C:\\Python\\Booking_22')
os.getcwd()


'C:\\Python\\Booking_22'

In [3]:
# загрузка библиотек
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta
import openpyxl
import xlsxwriter

import pymysql.cursors

import psycopg2
import pandas.io.sql as psql

In [4]:
# подключение к view из MTF booking
conn = psycopg2.connect(dbname='XXX', user='XXX', password='XXX', host='XX.XX.XX.XX')
cursor = conn.cursor()

cursor.execute('SELECT * FROM payments_view')

records = cursor.fetchall()

conn_2 = psycopg2.connect(dbname='XXX', user='XXX', password='XXX', host='XX.XX.XX.XX')
df = psql.read_sql('SELECT * FROM payments_view', conn_2)

df.info()

#  2021 10 22 грузилось 9,5 сек.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131659 entries, 0 to 131658
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   doc_num        131659 non-null  int64  
 1   doc_date       131659 non-null  object 
 2   cruise_type    131659 non-null  object 
 3   summ           131659 non-null  float64
 4   date_start     131659 non-null  object 
 5   date_finish    131659 non-null  object 
 6   route          131659 non-null  object 
 7   ship_name      131659 non-null  object 
 8   payer          131648 non-null  object 
 9   operator_name  131659 non-null  object 
 10  manager        131656 non-null  object 
 11  orderid        131659 non-null  int64  
 12  tourid         131659 non-null  int64  
 13  ptype          131659 non-null  object 
 14  office         117347 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 15.1+ MB


In [5]:
#преобразовываю в dstetime
df["doc_date"] = pd.to_datetime(df["doc_date"])
df["date_start"] = pd.to_datetime(df["date_start"])
df["date_finish"] = pd.to_datetime(df["date_finish"])

#добавляю год и месяц в отдельные столбцы
df['Month'] = df["doc_date"].dt.month
df['Year'] = df["doc_date"].dt.year
df['Day'] = df["doc_date"].dt.day


df['Month_start'] = df["date_start"].dt.month
df['Year_start'] = df["date_start"].dt.year


In [6]:
#столбец с тыс.₽
df["ST"] = df["summ"]/1000

#округлить df["ST"]:
df["ST2"] = pd.to_numeric(df["ST"]).round(1)

In [7]:
#Табл. данные по круизам только с 2021 года и далее
df1 = df[df["date_start"] >= '2022-01-01']

In [8]:
# нахождение крайней даты
lrd = df1["doc_date"].sort_values(ascending=True).tail(1).astype('datetime64[ns]')

#lrd
print(lrd)
print('-------')

#привоэу в более менее удобный формат для выдачи в текст заголовка
lrd2 = lrd.astype('str').values

print(lrd2)
print('-------')

lrd3 = lrd.astype('datetime64[ns]') #.dt.strftime('%Y/%m/%d')
print(lrd3)
print('-------')

41107   2021-11-01
Name: doc_date, dtype: datetime64[ns]
-------
['2021-11-01']
-------
41107   2021-11-01
Name: doc_date, dtype: datetime64[ns]
-------


In [9]:
# сегодня
today = pd.to_datetime('today')

print('today: ', today)

today2 = datetime.today().strftime('%Y-%m')

print('today2: ', today2)




today:  2021-11-02 18:21:01.370660
today2:  2021-11


In [10]:
from pandas.tseries.offsets import MonthEnd, MonthBegin

dbm = pd.to_datetime(today2)-MonthEnd(1)

print('dbm: ', dbm)

dbm:  2021-10-31 00:00:00


In [11]:
type(dbm)

pandas._libs.tslibs.timestamps.Timestamp

In [12]:
#Табл с данными только по МТФ теплоходам стартующими с 2022 года и далее
df1MTF = df1[df1['operator_name'] == "ОАО 'Мостурфлот'"]

In [13]:
#надо добавать столбцы с полож и отриц суммами в ST2
#df2020MTF.where(df2020MTF['ST2']>0) # верно отрабатывает

#plus = df2020MTF.where(df2020MTF['ST2']>0)
#minus = df2020MTF.where(df2020MTF['ST2']<0)

df1MTF['ST2p'] = np.where(df1MTF['ST2']>0, df1MTF['ST2'], 0) # верно отрабатывает
df1MTF['ST2m'] = np.where(df1MTF['ST2']<0, df1MTF['ST2'], 0) # верно отрабатывает

In [14]:
# база только за текущий месяц
# df1.info()

dftm = df1MTF[df1MTF['doc_date']>dbm]

# dftm.tail()

In [15]:
#все платежи (поступления и возвраты) по МТФ по теплоходам стартующими с 2021 года и далее
pt1 = pd.pivot_table(df1MTF, index = ['ship_name'], values = ["ST2"], columns = ['Year', 'Month'], aggfunc=np.sum, fill_value='', margins=True, margins_name='Всего')

pt1

ST2                                             
Year              2021                                        Всего
Month                7        8        9       10      11          
ship_name                                                          
Александр Грин            946.3   2172.0   1457.0            4575.3
Андрей РУБЛЕВ                      140.6   2582.0   167.0    2889.6
Василий СУРИКОВ                             413.5    86.9     500.4
И.А. КРЫЛОВ       71.4  12857.3   6422.5   4747.5           24098.7
Илья РЕПИН                                  922.8    38.6     961.4
Княжна ВИКТОРИЯ           261.0  12885.6  11536.6   232.9   24916.1
Михаил Булгаков         20443.5  11359.4  15538.9   704.6   48046.4
Николай КАРАМЗИН         4563.8   7034.4   5896.1   419.0   17913.3
СЕРГЕЙ ОБРАЗЦОВ          3107.8   4862.6   3641.4   169.9   11781.7
Сергей ЕСЕНИН                       68.8   5167.2   216.4    5452.4
Всего             71.4  42179.7  44945.9  51903.0  2035.3  141135.3

In [16]:
#все платежи (поступления и возвраты) по МТФ по теплоходам стартующими с 2021 года и далее  только за текущий месяц
pt1tm = pd.pivot_table(dftm, index = ['ship_name'], values = ["ST2"], columns = ['Year', 'Month', 'Day'], aggfunc=np.sum, fill_value='', margins=True, margins_name='Всего')

pt1tm

ST2        
Year                2021   Всего
Month                 11        
Day                    1        
ship_name                       
Андрей РУБЛЕВ      167.0   167.0
Василий СУРИКОВ     86.9    86.9
Илья РЕПИН          38.6    38.6
Княжна ВИКТОРИЯ    232.9   232.9
Михаил Булгаков    704.6   704.6
Николай КАРАМЗИН   419.0   419.0
СЕРГЕЙ ОБРАЗЦОВ    169.9   169.9
Сергей ЕСЕНИН      216.4   216.4
Всего             2035.3  2035.3

In [17]:
##все платежи (поступления и возвраты) по МТФ по теплоходам стартующими с 2021 года и далее по типам платежей

pt2 = pd.pivot_table(df1MTF, index = ['ptype'], values = ["ST2"], columns = ['Year', 'Month'], aggfunc=np.sum, fill_value='', margins=True, margins_name='Всего')

pt2

ST2                                     \
Year                             2021                                      
Month                               7        8        9       10      11   
ptype                                                                      
1.Касса в офисе                        13441.3  11856.2   9148.9           
2.Расчётный счёт(в осн. агенты)  71.4   1686.1   6208.7  15351.2  1657.6   
3.Банк. карта в офисе                  14247.9   9326.6   9938.2   187.6   
4.Онлайн оплата                         9070.8  13139.5  15080.5   183.4   
5.Переоформление(взаимозачёт)           3733.6   4414.9   2384.2     6.7   
Всего                            71.4  42179.7  44945.9  51903.0  2035.3   

                                           
Year                                Всего  
Month                                      
ptype                                      
1.Касса в офисе                   34446.4  
2.Расчётный счёт(в осн. агенты)   24975.0  
3.Банк. карта в офисе             33700.3  
4.Онлайн оплата                   37474.2  
5.Переоформление(взаимозачёт)     10539.4  
Всего                            141135.3

In [18]:
##все платежи (поступления и возвраты) по МТФ по теплоходам стартующими с 2021 года и далее по типам платежей  только за текущий месяц

pt2tm = pd.pivot_table(dftm, index = ['ptype'], values = ["ST2"], columns = ['Year', 'Month', 'Day'], aggfunc=np.sum, fill_value='', margins=True, margins_name='Всего')

pt2tm

ST2        
Year                               2021   Всего
Month                                11        
Day                                   1        
ptype                                          
2.Расчётный счёт(в осн. агенты)  1657.6  1657.6
3.Банк. карта в офисе             187.6   187.6
4.Онлайн оплата                   183.4   183.4
5.Переоформление(взаимозачёт)       6.7     6.7
Всего                            2035.3  2035.3

In [19]:
#все платежи (поступления и возвраты) по МТФ по теплоходам стартующими с 2021 года и далее
pt3 = pd.pivot_table(df1MTF, index = ['ship_name', 'date_start'], values = ["ST2"], columns = ['Year', 'Month'], aggfunc=np.sum, fill_value='', margins=True, margins_name='Всего')

pt3

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  


ST2                                             
Year                       2021                                        Всего
Month                         7        8        9       10      11          
ship_name      date_start                                                   
Александр Грин 2022-04-30                    67.3     70.6             137.9
               2022-05-03          581.5    564.8    299.2            1445.5
               2022-05-10                   147.9     51.0             198.9
               2022-05-16          151.6    160.2    310.5             622.3
               2022-05-29           56.1    131.7    188.0             375.8
...                         ...      ...      ...      ...     ...       ...
Сергей ЕСЕНИН  2022-07-18                            912.3    75.0     987.3
               2022-08-01                            441.2             441.2
               2022-08-27                    68.8    197.9             266.7
               2022-09-10                            111.0             111.0
Всего          NaT         71.4  42179.7  44945.9  51903.0  2035.3  141135.3

[152 rows x 6 columns]

In [20]:
# сегодня

today = pd.to_datetime('today')



In [23]:
today2 = datetime.today().strftime('%Y-%m-%d %H:%M')

today2

'2021-11-02 18:21'

In [24]:
txt_today = f"Отчет создан: {today2}"

txt_today

'Отчет создан: 2021-11-02 18:21'

In [25]:
txt1 = f"МТФ booking. Приток минус возвраты ДС по теплоходам стартующими с 2022 года (тыс.руб.).<br> Отчет на дату: {lrd2}"
txt2 = f"МТФ booking. Приток минус возвраты ДС по теплоходам стартующими с 2022 года (тыс.руб.).<br> Только за тукущий месяц по дням.<br> Отчет на дату: {lrd2}"
txt3 = f"МТФ booking. Приток минус возвраты ДС по теплоходам стартующими с 2022 года по типам платежей (тыс.руб.). <br> Отчет на дату: {lrd2}"
txt4 = f"МТФ booking. Приток минус возвраты ДС по теплоходам стартующими с 2022 года по типам платежей (тыс.руб.).<br> Только за тукущий месяц по дням. <br> Отчет на дату: {lrd2}"

txt4


"МТФ booking. Приток минус возвраты ДС по теплоходам стартующими с 2022 года по типам платежей (тыс.руб.).<br> Только за тукущий месяц по дням. <br> Отчет на дату: ['2021-11-01']"

In [26]:
# 2021 10 22

def write_to_html_file_1(df, title='', footer='', filename='out.html'):
    '''
    Write an entire dataframe to an HTML file with nice formatting.
    '''

    result = '''
<html>
<head>
<style>

    h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    color: #1919bc;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 0.5px solid black;
    border-collapse: collapse;
    border-color: #1919bc;
  }
  th, td {
    padding: 5px;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 80%;
    text-align: left;
    color: #1919bc;
    vertical-align: middle;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%;
  }

  thead > tr:nth-child(2) th:not(:first-child) {
    text-align: center;
  }
  tbody td {
    text-align: right;
  }



</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    result += df.to_html(escape=False) #classes='wide', float_format='{:.2f}'.format
    result += '''

<script>
  function numberWithSpaces(x) {
    var rounded = Math.floor(Number(x));
    var parts = rounded.toString().split(".");
    parts[0] = parts[0].replace(/\B(?=(\d{3})+(?!\d))/g, " ");
    return parts.join(".");
  }

  var items = document.querySelectorAll('tbody td');

  items.forEach(function(item) {
    var formattedNumber = numberWithSpaces(item.textContent);
    if(item.textContent === '') {
      return;
    }
    item.textContent = formattedNumber;
  });
</script>


</body>

<center>
	<footer>
      <br />
      <br />
      <b>Создал:</b> <i>Смоленышев Олег В.</i>
      <br />
      <a href="mailto:smolenyshev@gtm-one.com">smolenyshev@gtm-one.com</a>
      <br />
      '''
    result += '<i> %s </i>\n' % footer
    result += '''
  </footer>
</center>


</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

In [27]:
# 2021 10 22 создаю html

write_to_html_file_1(pt1, txt1, txt_today, 'МТФ_платежи_навигация_2022_теплоходы.html')
write_to_html_file_1(pt1tm, txt2, txt_today, 'МТФ_платежи_навигация_2022_теплоходы_текущ_мес.html')
write_to_html_file_1(pt2, txt3, txt_today, 'МТФ_платежи_навигация_2022_виды.html')
write_to_html_file_1(pt2tm, txt4, txt_today, 'МТФ_платежи_навигация_2022_виды_текущ_мес.html')

In [28]:
#пакеты для работы с почтой
import smtplib
import mimetypes
import email
import email.mime.application
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

In [33]:
# %%
text_mail1 = "МТФ-bppking. Навигация 2022"

text_mail2 = "Здравствуйте, <br> Продажи туров на теплоходы наигации 2022 года. Данные из системы  МТФ-booking.<br> Отчет будет запускаться каждый раб.день с 1 ноября 2021 в 9:45. <br> <br>                 С уважением,<br> Смоленышев Олег <br>                 smolenyshev@mosrp.ru <br>                 +7 916 014 37 70"
                
print(text_mail2)

Здравствуйте, <br> Продажи туров на теплоходы наигации 2022 года. Данные из системы  МТФ-booking.<br> Отчет будет запускаться каждый раб.день с 1 ноября 2021 в 9:45. <br> <br>                 С уважением,<br> Смоленышев Олег <br>                 smolenyshev@mosrp.ru <br>                 +7 916 014 37 70


In [ ]:
# но в почте выглядит не так как тут

text_mail3 = f"Здравствуйте, \n\
    Продажи туров на теплоходы наигации 2022 года. Данные из системы  МТФ-booking.<br>\n\
    Отчет будет запускаться каждый раб.день с 1 ноября 2021 в 9:45. <br> <br>\n\
    {pt1} <br> <br>\n\
    {pt2} <br> <br> \n\
    {pt1tm} <br> <br> \n\
    {pt2tm} <br> <br> \n\
    С уважением,<br> Смоленышев Олег <br>                 smolenyshev@mosrp.ru <br>                 +7 916 014 37 70"
                
print(text_mail3)

In [45]:
a = ['smolenyshev@gtm-one.com']

receipts = ('Получатели: ', a)

print(receipts)

('Получатели: ', ['smolenyshev@gtm-one.com'])


In [46]:
# %%

a = ['smolenyshev@gtm-one.com']

msg = MIMEMultipart()
msg['Subject'] = text_mail1
msg['From'] = 'syslog2@mosrp.ru'
msg['To'] = 'smolenyshev@gtm-one.com'

body = MIMEText(text_mail3, 'html') # text_mail2

msg.attach(body)

dir_path = 'C:\\Python\\Booking_22'
files = ["МТФ_платежи_навигация_2022_теплоходы.html", "МТФ_платежи_навигация_2022_теплоходы_текущ_мес.html", "МТФ_платежи_навигация_2022_виды.html", "МТФ_платежи_навигация_2022_виды_текущ_мес.html"] #, 'mrp_CF_graf.html', 'mrp_pritok_day.html', 'mrp_ottok_day.html'

for f in files:  # add files to the message
        file_path = os.path.join(dir_path, f)
        att = MIMEApplication(open(file_path, "rb").read(), _subtype="html")
        att.add_header('Content-Disposition','attachment', filename=f)
        msg.attach(att)


s = smtplib.SMTP_SSL('XXX.XXX.XX', 465)
s.login('XXX', 'XXX')



for i in range(0, len(a)):
    s.sendmail('syslog2@mosrp.ru', a[i], msg.as_string())

# print("Отправлен отчет МТФ по ДС")

print(a, files, sep='\n')

s.quit()

['smolenyshev@gtm-one.com']
['МТФ_платежи_навигация_2022_теплоходы.html', 'МТФ_платежи_навигация_2022_теплоходы_текущ_мес.html', 'МТФ_платежи_навигация_2022_виды.html', 'МТФ_платежи_навигация_2022_виды_текущ_мес.html']


(221, b'2.0.0 SMTP closing connection')